# Fine-Tuning LLMs with Hugging Face

## Step 1: Installing and importing the libraries

In [ ]:
!pip uninstall accelerate peft bitsandbytes transformers trl -y
!pip install accelerate peft==0.13.2 bitsandbytes transformers trl==0.12.0

Found existing installation: accelerate 1.3.0
Uninstalling accelerate-1.3.0:
  Successfully uninstalled accelerate-1.3.0
Found existing installation: peft 0.13.2
Uninstalling peft-0.13.2:
  Successfully uninstalled peft-0.13.2
Found existing installation: bitsandbytes 0.45.2
Uninstalling bitsandbytes-0.45.2:
  Successfully uninstalled bitsandbytes-0.45.2
Found existing installation: transformers 4.48.3
Uninstalling transformers-4.48.3:
  Successfully uninstalled transformers-4.48.3
Found existing installation: trl 0.12.0
Uninstalling trl-0.12.0:
  Successfully uninstalled trl-0.12.0
  Using cached accelerate-1.3.0-py3-none-any.whl.metadata (19 kB)
  Using cached peft-0.13.2-py3-none-any.whl.metadata (13 kB)
  Using cached bitsandbytes-0.45.2-py3-none-manylinux_2_24_x86_64.whl.metadata (5.8 kB)
  Using cached transformers-4.48.3-py3-none-any.whl.metadata (44 kB)
  Using cached trl-0.12.0-py3-none-any.whl.metadata (10 kB)
Using cached peft-0.13.2-py3-none-any.whl (320 kB)
Using cached tr

In [ ]:
!pip install huggingface_hub

In [ ]:
import torch
from trl import SFTTrainer
from peft import LoraConfig
from datasets import load_dataset
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline)

## Step 2: Loading the model

In [ ]:
# load a pre-trained model for casual language modeling
llama_model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path="aboonaji/llama2finetune-v2",
    quantization_config=BitsAndBytesConfig( # apply quantization to the model to reduce its memory usage
        load_in_4bit=True, # specify that the model should be loaded in 4-bit precision instead of the default 32-bit floating point
        bnb_4bit_compute_dtype=getattr(torch, "float16"), # 16-bit floating point), which is a good balance between precision and performance.
        bnb_4bit_quant_type="nf4" # "nf4" stand for NormalFloat 4-bit, a quantization method optimized for neural networks.
    )
)
llama_model.config.use_cache = False # disable this caching mechanism -> reduce memory usage during inference, especially for long sequences, but may slightly slow down generation.
llama_model.config.pretraining_tp = 1 # disable tensor parallelism, 1 is chosen when running the model on a single GPU or CPU

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Step 3: Loading the tokenizer

In [ ]:
llama_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path = "aboonaji/llama2finetune-v2", trust_remote_code = True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token # set the padding token to be the same as the end-of-sequence token
llama_tokenizer.padding_side = "right" # configure the padding to occur on the right side of the sequence.

## Step 4: Setting the training arguments

In [ ]:
training_arguments = TrainingArguments(output_dir = "./results", per_device_train_batch_size = 4, max_steps = 100)

## Step 5: Creating the Supervised Fine-Tuning trainer

In [ ]:
llama_sft_trainer = SFTTrainer(model = llama_model,
                               args = training_arguments,
                               train_dataset = load_dataset(path = "aboonaji/wiki_medical_terms_llam2_format", split = "train"),
                               tokenizer = llama_tokenizer,
                               peft_config = LoraConfig(task_type = "CAUSAL_LM", r = 64, lora_alpha = 16, lora_dropout = 0.1),
                               dataset_text_field = "text")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


## Step 6: Training the model

In [ ]:
batch_size = 8  # or some smaller value

In [ ]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [ ]:
llama_sft_trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: rimnguyencnn (rimnguyencnn-futurice) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


OutOfMemoryError: CUDA out of memory. Tried to allocate 172.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 132.12 MiB is free. Process 124931 has 14.61 GiB memory in use. Of the allocated memory 13.94 GiB is allocated by PyTorch, and 562.88 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

## Step 7: Chatting with the model

In [ ]:
user_prompt = "Please tell me about Bursitis"
text_generation_pipeline = pipeline(task = "text-generation", model = llama_model, tokenizer = llama_tokenizer, max_length = 500)
model_answer = text_generation_pipeline(f"<s>[INST] {user_prompt} [/INST]")
print(model_answer[0]['generated_text'])